In [ ]:
##Programa para calcular la correlación de una serie temporal con el método de DFA. La serie temporal ha sido transformada a
## partir de una secuencia de nucleotidos siguiendo una regla de mapeo binaria. Para formar esta serie temporal tenemos que 
## recorrer la secuencia y si encontramos una Adenina(A), una Timina(T) o Uracilo(U) el valor de la serie temporal crece y si 
## encontramos una Citosina (C) o una Guanina (G) el valor de la serie decrece. Es una forma distinta de trasnformar las secuencias 
## de nucleotidos que la que propuso Peng et al. en 1992 [Long-range correlations in nucleotide sequences. Nature.], basada en la complementaridad de estos nucleotidos en las cadenas 
## de doble helice, en vez de la ocmposición química de los compuestos.

##PAQUETES

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
# from Bio.SeqRecord import SeqRecord
# import time
# from scipy.optimize import curve_fit
# from graph_tools import Graph
# import random
from scipy import stats
# import math
# import scipy.optimize as spo
import datetime


## Transformar el formato .fasta (formato en el que se descargan las secuencias del NCBI) a una string con la secuencia de nucleotidos

def FastaToSeq(DNA_file):
    file_in =DNA_file

    #file_out='gene_seq_out.fasta'

    fin = open(file_in,mode='r')
    for record in SeqIO.parse(fin, 'fasta'):
        rID = record.id
        rDESC = record.description
        rSEQ = record.seq
    rDESC_2 = ' '.join(rDESC.split()[1:])
    print('SequenceID = '  + rID)
    print('Description = ' + rDESC_2 + '\n')
    # write new fasta file
    #r=SeqIO.write(seq_record, f_out, 'fasta')
    #if r!=1: print('Error while writing sequence:  ' + seq_record.id)
    DNA_seq= rSEQ
    DESC = rDESC_2
    return DNA_seq,DESC,rID


##Ubicación de la carpeta con las secuencias a analizar

folder = './SecsGRANDES'

## Dataframe donde almacenar los datos 

columnas1 = ["Identificador",
            'NombreSecuencia',
           'LongitudSecuencia',
            'PENG-alpha',
            'PENG-R2'
           ]

dtt_Peng = pd.DataFrame(columns=columnas1)

## Lista con los ficheros .fasta que vamos a analizar 
listaFicheros=[]
for root, dirs, files in os.walk(folder):
    for file in files:
        #print(root,dirs,file)
        if file.endswith('.fasta') :
            listaFicheros.append([os.path.join(root,file)])
            
print('Long lista ficheros: ',len(listaFicheros))

#R2 que queremos para nuestro análisis
R2paraPeng=0.995

## Creamos el DNA WALK y  calculamos su correlación 
for iii,ff in enumerate(listaFicheros):

    now = datetime.datetime.now()

    print('Tiempo: ',now.strftime("%m/%d/%Y, %H:%M:%S"))
    print(iii,ff)
    print(ff[0])


    DNA_seq,dscrp,rID = FastaToSeq(ff[0])
    DNA_seq= "O" + DNA_seq
    length_seq=len(DNA_seq)
    ##DNA WALK
    DNA_walk=[]
    x=np.arange(length_seq)
    y=0
    for i in range(length_seq):
        if DNA_seq[i] in ["C","G"]:
            y=y -1
        elif DNA_seq[i] in ["T","A","U"]:
            y=y +1
        else:
            y=y

        DNA_walk.append(y)
        
    F=[]
    L=[] #log de L
    # valores permisibles de l, serán:
    lposibles = np.arange(1,int(np.floor(length_seq/1.5)+1))
    #calculamos la correlación 
    # F^2(l) 
    for l in lposibles:
        fluct2 = 0
        Delta_y = []
        Delta_y_2 = []
        for l0 in range(0,length_seq-l):
            Delta_y.append((DNA_walk[l0+l]-DNA_walk[l0]))

            Delta_y_2.append((DNA_walk[l0+l]- DNA_walk[l0])**2)

        Delta_y_mean = np.mean(Delta_y)

        Delta_y_2_mean = np.mean(Delta_y_2)


        fluct2 = Delta_y_2_mean-Delta_y_mean**2


        ff = np.sqrt(fluct2)
        if ff==0:
            F.append(0)
        else:
            F.append(np.log10(ff))


    L = np.log10(lposibles)


    slope_list=[]
    R2=[]
    Lmaxposibles=[]
    Nposibles=np.arange(30,1,-0.5)
    for i in Nposibles:
        Lmax=int(np.floor(length_seq/i))
        Lsub=L[0:Lmax]
        Fsub=F[0:Lmax]
        slope, intercept, r_value, p_value, std_err = stats.linregress(Lsub,Fsub)
        R2_value=r_value**2
        Lmaxposibles.append(Lmax)


        slope_list.append(slope)
        R2.append(R2_value)
        

    #Seleccionamos el resultado con el valor de R2 que queremos obtener
    indice = np.argmin(np.abs(np.asarray(R2)-R2paraPeng))
    R2obtenido = R2[indice]
    Noptimo = Nposibles[indice]
    SlopeOptimo = slope_list[indice]
    
    
    dtt_Peng=dtt_Peng.append({'Identificador':rID,'NombreSecuencia':dscrp,'LongitudSecuencia':length_seq,'PENG-alpha':SlopeOptimo,"PENG-R2":R2obtenido},ignore_index=True)
#Guardamos los resultados
print(dtt_Peng)
dtt_Peng.to_csv(path_or_buf="Resultados_correlacion_DNAWALKS.csv", sep=";")
    

Long lista ficheros:  5
Tiempo:  10/03/2021, 23:17:10
0 ['./SecsGRANDES\\BRCA1_human_gene.fasta']
./SecsGRANDES\BRCA1_human_gene.fasta
SequenceID = NG_005905.2
Description = Homo sapiens BRCA1 DNA repair associated (BRCA1), RefSeqGene (LRG_292) on chromosome 17

Tiempo:  10/04/2021, 07:00:12
1 ['./SecsGRANDES\\EGFR_Human.fasta']
./SecsGRANDES\EGFR_Human.fasta
SequenceID = NG_007726.3
Description = Homo sapiens epidermal growth factor receptor (EGFR), RefSeqGene (LRG_304) on chromosome 7

Tiempo:  10/04/2021, 19:06:03
2 ['./SecsGRANDES\\ESR1_human.fasta']
./SecsGRANDES\ESR1_human.fasta
SequenceID = NG_008493.2
Description = Homo sapiens estrogen receptor 1 (ESR1), RefSeqGene (LRG_992) on chromosome 6

